In [1]:
import numpy as np

# What is this?

1. This is an auction making project
2. Three components
  1. User
    1. User gets its proba when `__init__`
    2. based on this proba, `show_ad` is decided
  2. Bidder
    1. Buyer of rights to (try to) show ad to selected user
    2. Highest bidder wins
      1. Winner gets to show ad to selected user
      2. IF user clicks, winner gets $1
      3. REGARDLESS of click, winner loses second highest bid
      

---
## User

In [150]:
class User:
    def __init__(self):
        self.__p = self.__probability = np.random.random()
    
    def show_ad(self):
        return np.random.choice([True, False], 
                                p=(self.__p, 1 - self.__p))
    
    def show_proba(self):
        # custom method
        return self.__p
       

---
## Auction

In [ ]:
# enumerate: create index (default: starting from 0)

In [117]:
class Auction:
    def __init__(self, users: list, bidders: list):
        self.users = users
        self.bidders = bidders
        self.uids = {user: uid for uid, user in enumerate(users)}
        self.balances = {b: b.balance for b in bidders}
        
    def execute_round(self):
        # Select one user at random
        user = np.random.choice(self.users)
        # Then query user's id from `self.uids` dictionary
        uid = self.uids[user]
        
        # Bidders make bids based on bid
        bids = [bidder.bid(uid) for bidder in self.bidders]
        
        # Find maximum bidding
        max_bid = max(bids)
        
        # Find maximum bidders (there can be more than 1)
        max_bidder_ids = [n for n, b in enumerate(bids) if b == max_bid]
        winner = np.random.choice(max_bidder_ids)
        
        # Get second highest price
        # sorted: lowest -> highest by default
        winning_price = sorted(bids)[-2]
        
        for n, bidder in enumerate(self.bidders):
            if n == winner:
                bidder.notify(auction_winner=True,
                              price=winning_price,
                              clicked=user.show_ad())
                
                # Update bidders balance
                # Balance is changed to winning bidder only
                self.balances[bidder] = bidder.balance
            
            else:
                bidder.notify(auction_winner=False,
                              price=winning_price,
                              clicked=None)    
    
    def plot_history(self):
        # Optional credit
        pass

---

## Bidder

In [118]:
class Bidder:
    def __init__(self, num_users, num_rounds):
        self.num_users = num_users
        self.num_rounds = num_rounds
        self.balance = 0
        
    def bid(self, user_id):
        # This is the most important part
        # Bidder must learn user and predict probability
        return round(np.random.random(), 3)
    
    def notify(self, 
               auction_winner: bool, 
               price: float, 
               clicked: bool):
        if auction_winner:
            if clicked:
                self.balance += 1
                
            self.balance -= price
        
        ## Even if they lose, they get to know price (second higest price)
        
    
    def is_disq(self):
        # custom method to check if bidder is disqualified
        return self.balance < -1000

In [127]:
## End of 80% credit
b0, b1, b2 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction([User()],[b0, b1, b2])
auction.execute_round()
bal = auction.balances

In [128]:
## Testing bidder's smartness

In [135]:
winner_cnt = defaultdict(int)
n_auctions = 1000
n_rounds = 100
n_bidder = 5

In [133]:
for _ in range(n_auctions):
    users = [User() for _ in range(np.random.randint(2, 11))]
    auc = Auction(users, [*(Bidder(len(users), n_rounds) for _ in range(n_bidder))])
    for i in range(n_rounds):
        auc.execute_round()
    winner_cnt[np.argmax(list(auc.balances.values()))] += 1

In [134]:
{k: winner_cnt[k] for k in sorted(winner_cnt)}

{0: 196, 1: 184, 2: 194, 3: 214, 4: 212}

---
## KingBidder

Optional Credit

In [196]:
from collections import defaultdict

class KingBidder:
    def __init__(self, num_users, num_rounds):
        self.num_users = num_users
        self.num_rounds = num_rounds
        self.balance = 0
        self.data = defaultdict(list)
        self.cur_user = None
        
    def bid(self, user_id):
        # This is the most important part
        # Bidder must learn user and predict probability
        self.cur_user = user_id
        proba = 0.5
        if (data:= self.data[user_id]):
            proba = sum(data) / len(data)
        return 100 * np.random.random() * np.random.choice([True, False], 
                                                           p=(proba, 1-proba))
    
    def notify(self, 
               auction_winner: bool, 
               price: float, 
               clicked: bool):
        if auction_winner:
            if clicked:
                self.balance += 1
            self.data[self.cur_user].append(clicked)
            self.balance -= price
        
    
    def is_disq(self):
        return self.balance < -1000

In [136]:
winner_cnt = defaultdict(int)
n_auctions = 1000
n_rounds = 100
n_bidder = 5

In [ ]:
for _ in range(n_auctions):
    users = [User() for _ in range(np.random.randint(2, 11))]
    auc = Auction(users, [KingBidder(len(users), n_rounds), 
                          *(Bidder(len(users), n_rounds) for _ in range(n_bidder-1))])
    for i in range(n_rounds):
        auc.execute_round()
    winner_cnt[np.argmax(list(auc.balances.values()))] += 1

In [138]:
{k: winner_cnt[k] for k in sorted(winner_cnt)}

{0: 249, 1: 186, 2: 197, 3: 185, 4: 183}

In [142]:
auc.bidders[0].data

defaultdict(list,
            {3: [True, False, False, True, False, True, True],
             1: [True, False, True, False, False, False, False, False],
             2: [False],
             0: [False],
             4: [True,
              False,
              True,
              True,
              False,
              True,
              False,
              True,
              False,
              False,
              False],
             5: [False]})

In [144]:
for u in auc.users:
    print(u.show_proba())

0.5056451160807595
0.590660304734246
0.1254374551804669
0.3676688734706821
0.6353125323356017
0.15487911947757949
